
# <p style="text-align:center"> <font color='darkorange'>**CUNEF**</font>
## <p style="text-align:center"> **TFM - Análisis de sentimiento pólitico en Twitter**
### <p style="text-align:center"> **3. Predicciones con GPT3**</strong><br />
    
<p style="text-align:left">Pablo Mazariegos Reviriego - <font color='orange'>pablo.mazariegos@cunef.edu </font>

En este proyecto de Trabajo Fin de Máster, realizaremos un análisis de sentimiento de los tweets hechos por los 5 candidatos políticos a la presidencia de Madrid durante el período de campaña política que abarcó desde el 12 hasta el 27 de mayo de 2023. Utilizaremos una base de datos recopilada manualmente que contiene los tweets de los candidatos. El objetivo principal de este proyecto es desarrollar modelos de aprendizaje automático que puedan clasificar los tweets según su sentimiento (positivo, negativo o neutral).

El proyecto se organizará en diferentes cuadernos, cada uno enfocado en una etapa específica del proceso:

 1. EDA y Selección/Preparación de variables
 2. Word Cloud y Análisis de viralidad
 3. <font color='darkgreen'> **Predicciones con GPT3**</font>
 4. Otros Modelos

Este cuaderno se enfoca 

  **INDICE:**
 
 - [Importación de Librerias](#0) 
 - [Funciones utilizadas en este notebook](#1) 
 - [Carga de datos](#2)
 - [Creación de Json de los datos](#3)

##  <a name="0"> Importación de Librerias</a>

In [41]:
import pandas as pd
import seaborn as sns
import json
import openai
from getpass import getpass

import warnings
warnings.filterwarnings('ignore')

##  <a name="2"> Carga de datos</a>

In [ ]:
API_KEY_OPENAI = getpass('Introduce tu API_KEY de Open AI:')

openai.api_key = API_KEY_OPENAI
os.environ['OPENAI_API_KEY'] = API_KEY_OPENAI

In [20]:
file_path = '../data/processed/df_sentimiento_final.csv'
df = pd.read_csv(file_path)
df = df.drop("Unnamed: 0", axis=1)
df.head()

,partido,partido_num,candidato,nick,followers,fecha,post,post_clean,sentimiento,sentimiento_num,...,comments vs viewed,shared,shared vs viewed,likes,likes vs viewed,votos,porcentaje,escaños,Num_words_POST,sentimiento_clean
0,PP,4,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Comenzamos la campaña, una vez más, junto a la...",comenzamos la campaa una vez ms junto a la agr...,Positive,1,...,0.003373,261,0.004446,1260,0.021465,1586985,0.4734,71,42,comenzamos campaa vez ms junto agricultura cam...
1,PP,4,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Madrid es la región del Espíritu de Ermua, la ...",madrid es la regin del espritu de ermua la que...,Negative,-1,...,0.005413,561,0.005522,2127,0.020935,1586985,0.4734,70,24,madrid regin espritu ermua admite totalitarios...
2,PP,4,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Majadahonda con ganas de Libertad, familia, un...",majadahonda con ganas de libertad familia unid...,Positive,1,...,0.002345,213,0.003568,1042,0.017454,1586985,0.4734,70,18,majadahonda ganas libertad familia unidad aleg...
3,PP,4,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,Presidente: líbranos del mal.,presidente lbranos del mal,Negative,-1,...,0.002700,549,0.001660,2592,0.007836,1586985,0.4734,70,4,presidente lbranos mal
4,PP,4,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,En Madrid sólo habrá 2 tarifas de transporte p...,en madrid slo habr 2 tarifas de transporte pbl...,Negative,-1,...,0.001445,759,0.003562,3016,0.014153,1586985,0.4734,70,51,madrid slo habr 2 tarifas transporte pblico ma...


##  <a name="3"> Creación de Json de los datos</a>

## Guardamos los tweets y partido en un Json

In [24]:
# Crea un nuevo data frame con las columnas que necesitas
new_df = df[['post_clean', 'partido']]

# Crea una lista vacía para almacenar los nuevos datos
data = []

# Itera sobre cada fila del nuevo data frame
for index, row in new_df.iterrows():
    # Crea un diccionario con el formato que necesitas
    entry = {"prompt": row['post_clean'], "completion": row['partido']}
    # Añade el diccionario a la lista
    data.append(entry)

# Finalmente, escribe los datos en un archivo JSON
with open('formatted_tweets_politicos.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


## Guardamos los tweets y sentimiento en un Json

In [27]:
# Crea un nuevo data frame con las columnas que necesitas
new_df = df[['post_clean', 'sentimiento']]

# Crea una lista vacía para almacenar los nuevos datos
data = []

# Itera sobre cada fila del nuevo data frame
for index, row in new_df.iterrows():
    # Crea un diccionario con el formato que necesitas
    entry = {"prompt": row['post_clean'], "completion": row['sentimiento']}
    # Añade el diccionario a la lista
    data.append(entry)

# Finalmente, escribe los datos en un archivo JSON
with open('formatted_tweets_sentimiento.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)


In [36]:
!openai tools fine_tunes.prepare_data -f formatted_tweets_politicos.json -q

Analyzing...

- Your file contains 770 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 5 duplicated prompt-completion sets. These are rows: [117, 425, 526, 539, 592]
- More than a third of your `completion` column/key is uppercase. Uppercase completions tends to perform worse than a mixture of case encountered in normal language. We recommend to lower case the data if that makes sense in your domain. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completio

C:\Users\Pablo\anaconda3\lib\site-packages\openai\validators.py:452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[column] = x[column].str.lower()
C:\Users\Pablo\anaconda3\lib\site-packages\openai\validators.py:226: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["prompt"] += suffix
C:\Users\Pablo\anaconda3\lib\site-packages\openai\validators.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [38]:
!openai api fine_tunes.create -t "formatted_tweets_politicos_train.jsonl" -v "formatted_tweets_politicos_valid.jsonl" -m text-davinci-003

Error: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.


In [37]:
!openai api fine_tunes.create -t formatted_tweets_politicos_prepared.jsonl -m text-davinci-003

Error: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.


In [ ]:
#openai tools fine_tunes.prepare_data -f <formatted_tweets_sentimiento.json> -q

In [ ]:
penai api fine_tunes.create -t  <LOCAL_FILE>_prepared.jsonl -m text-davinci-003